# Data-Pipelines 📊

## Context and objectives

### The aim of this project is to build a data pipeline to enrich an existing public dataset. For this goal, multiple public datasets will be combined together with a database scrapped from a website.


#### As we all know, there is a growing tendency to purchase electric cars. Not only they are becoming popular but also governments are fostering it by providing aids for people to purchase electric cars.

#### This project goal is to democratize data about electric cars evolution and its enviromental and health impac, if any.


#### The chosen scrapped dataset was "World-most-polluted-countries" from iqair.com. The scrapped table provides historical data about the most polluted country and region ranking based on annual average PM2.5 concentration (μg/m³).

####  In order to enrich this dataset, 3 other indicators will be considered and added: 1) charging points around the world, 2) sales cars history, 3) EV stock share and 4) number of deaths by risk factor

The following hypotheses were formulated to guide the analysis:

- The more charging stations, the higher the sales.
- The higher stock share, the higher the sales.
- The higher the sales, the less deaths.
- The more sales, the less the pollution.





## Libraries

In [139]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import pycountry
import pycountry_convert as pc
import country_converter as coco

In [ ]:
chargers=pd.read_csv("data/IEA-EV-dataEV charging pointsEVHistorical.csv")
sales=pd.read_csv("data/IEA-EV-dataEV salesCarsHistorical.csv") 
stock=pd.read_csv("data/IEA-EV-dataEV stock shareCarsHistorical.csv") 
deaths=pd.read_csv("data/number-of-deaths-by-risk-factor.csv") 


In [4]:
url = "https://www.iqair.com/world-most-polluted-countries"

In [114]:
html= requests.get(url)
pd.set_option('display.max_rows', 10)


In [5]:
soup= BeautifulSoup(html.content, "html.parser")

In [7]:
results2=soup.find_all("span", attrs = {"class":"text-normal"})

In [127]:
    url = "https://www.iqair.com/world-most-polluted-countries"
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    results=soup.find_all("div", attrs = {"class": "inner-table"})
    pd.set_option('display.max_rows', 10)
    df=pd.read_html(results[0].prettify())[0]
    df

,Rank,Country/Region,2022,2021,2020,2019,2018,Population
0,1,Chad,89.7,75.9,-,-,-,17179740
1,2,Iraq,80.1,49.7,-,39.6,-,43533592
2,3,Pakistan,70.9,66.8,59,65.8,74.3,231402117
3,4,Bahrain,66.6,49.8,39.7,46.8,59.8,1463265
4,5,Bangladesh,65.8,76.9,77.1,83.3,97.1,169356251
...,...,...,...,...,...,...,...,...
126,127,"Bonaire, Saint Eustatius and Saba",3.3,5.1,-,-,-,26221
127,128,Bermuda,3.0,-,-,-,-,63867
128,129,U.S. Virgin Islands,2.9,4.5,3.7,3.5,-,105870
129,130,French Polynesia,2.5,-,-,-,-,304032


In [147]:
pollution_2 = pd.melt(df, id_vars=['Country/Region', 'Population'], value_vars=['2018', '2019', '2020', '2021', '2022'], var_name='Year', value_name='Pollution')

pd.set_option('display.max_rows', 10)
pollution_2.rename(columns = {"Country/Region": "region", "Population":"population", "Pollution":"pollution", "Year":"year"}, inplace=True)
pollution_2.drop(pollution_2[pollution_2['pollution'] == '-'].index, inplace = True)
pollution_2.dropna()

pollution_2['year'] = pollution_2['year'].astype(int)
pollution_2['pollution'] = pollution_2['pollution'].astype(float)

pollution_2




,region,population,year,pollution
2,Pakistan,231402117,2018,74.3
3,Bahrain,1463265,2018,59.8
4,Bangladesh,169356251,2018,97.1
6,Kuwait,4250114,2018,56.0
7,India,1407563842,2018,72.5
...,...,...,...,...
650,"Bonaire, Saint Eustatius and Saba",26221,2022,3.3
651,Bermuda,63867,2022,3.0
652,U.S. Virgin Islands,105870,2022,2.9
653,French Polynesia,304032,2022,2.5


In [130]:
chargers_2= chargers.iloc[:, [0,5, 7]]


In [164]:

sales_2= sales.iloc[:, [0,5, 7]]
sales_2.rename(columns = {"value":"sales"}, inplace=True)


sales_2

,region,year,sales
0,Australia,2011,49
1,Australia,2012,170
2,Australia,2012,80
3,Australia,2013,100
4,Australia,2013,190
5,Australia,2014,370
6,Australia,2014,950
7,Australia,2015,1000
8,Australia,2015,760
9,Australia,2016,670


In [165]:
stock_2= stock.iloc[:, [0,5, 7]]
stock_2.rename(columns = {"value":"stock price %"}, inplace=True)
stock_2

,region,year,stock price %
0,Australia,2011,0.000460
1,Australia,2012,0.002800
2,Australia,2013,0.005400
3,Australia,2014,0.017000
4,Australia,2015,0.032000
5,Australia,2016,0.043000
6,Australia,2017,0.061000
7,Australia,2018,0.089000
8,Australia,2019,0.160000
9,Australia,2020,0.220000


In [86]:
deaths_2= deaths.iloc[:, [0,2, 3]]


In [137]:
deaths_2.rename(columns = {"Entity": "region", "Deaths - Cause: All causes - Risk: Outdoor air pollution - OWID - Sex: Both - Age: All Ages (Number)":"number of deaths by air pollution", "Year":"year"}, inplace=True)
deaths_2

,region,year,number of deaths by air pollution
0,Afghanistan,1990,3169
1,Afghanistan,1991,3222
2,Afghanistan,1992,3395
3,Afghanistan,1993,3623
4,Afghanistan,1994,3788
...,...,...,...
6835,Zimbabwe,2015,2835
6836,Zimbabwe,2016,2781
6837,Zimbabwe,2017,2700
6838,Zimbabwe,2018,2669


In [167]:
merged_table = pollution_2.merge(stock_2,on=['region', 'year']).merge(deaths_2,on=['region', 'year']).merge(sales_2,on=['region', 'year'])
pd.set_option('display.max_rows', 1000)
merged_table


,region,population,year,pollution,stock price %,number of deaths by air pollution,sales
0,India,1407563842,2018,72.5,0.0220,1093823,920
1,China,1412360000,2018,41.2,1.1000,1467298,270000
2,China,1412360000,2018,41.2,1.1000,1467298,820000
3,Chile,19493184,2018,24.9,0.0130,5631,110
4,Chile,19493184,2018,24.9,0.0130,5631,68
5,Mexico,126705138,2018,20.3,0.0190,37266,1600
6,Mexico,126705138,2018,20.3,0.0190,37266,200
7,Italy,59109668,2018,14.9,0.0590,27470,4700
8,Italy,59109668,2018,14.9,0.0590,27470,5000
9,Israel,9364000,2018,18.6,0.2000,2427,130


In [166]:
from functools import reduce
data_frames = [pollution_2, sales_2, deaths_2, stock_2]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['region', 'year'],
                                            how='outer'), data_frames)
df_merged

,region,population,year,pollution,sales,number of deaths by air pollution,stock price %
0,Pakistan,2.314021e+08,2018,74.3,NaN,118845.0,NaN
1,Bahrain,1.463265e+06,2018,59.8,NaN,598.0,NaN
2,Bangladesh,1.693563e+08,2018,97.1,NaN,77886.0,NaN
3,Kuwait,4.250114e+06,2018,56.0,NaN,1476.0,NaN
4,India,1.407564e+09,2018,72.5,920.0,1093823.0,0.022
...,...,...,...,...,...,...,...
7667,Zimbabwe,NaN,2015,NaN,NaN,2835.0,NaN
7668,Zimbabwe,NaN,2016,NaN,NaN,2781.0,NaN
7669,Zimbabwe,NaN,2017,NaN,NaN,2700.0,NaN
7670,Zimbabwe,NaN,2018,NaN,NaN,2669.0,NaN
